In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

import seaborn as sns
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

from numpy import random
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score

import warnings
warnings.simplefilter("ignore")



To bring in the code from the 'from learntools.time_series.style import * command from the Kaggle learntools components, since it was very difficult to actually install learntools on a computer. 

In [2]:
plt.style.use("seaborn-whitegrid")
plt.rc(
    "figure",
    autolayout=True,
    figsize=(11, 4),
    titlesize=18,
    titleweight='bold',
)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)

get_ipython().config.InlineBackend.figure_format = 'retina'

To create the seasonal_plot function from the Kaggle learntools files.

In [3]:
def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette(
        "husl",
        n_colors=X[period].nunique(),
    )
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax


To create the plot_periodograms function from the Kaggle learntools files.

In [4]:
def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

In [5]:
test = pd.read_csv('test_features.csv')
train = pd.read_csv('train_features.csv')
holidays = pd.read_csv('holidays_events.csv')
oil = pd.read_csv('oil.csv')
stores = pd.read_csv('stores.csv')
transactions = pd.read_csv('transactions.csv')
submission = pd.read_csv('sample_submission.csv')

In [6]:
submission

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


In [7]:
test

,id,date,store_nbr,family,onpromotion,day_of_week,weekend,year,month,day_of_month,...,city,state,type,cluster,date_str,national_holiday,national_event,national_workday,local_holiday,regional_holiday
0,3000888,2017-08-16,1,AUTOMOTIVE,0,3,0,2017,8,16,...,Quito,Pichincha,D,13,2017-08-16,0,0,0,0,0
1,3000889,2017-08-16,1,BABY CARE,0,3,0,2017,8,16,...,Quito,Pichincha,D,13,2017-08-16,0,0,0,0,0
2,3000890,2017-08-16,1,BEAUTY,2,3,0,2017,8,16,...,Quito,Pichincha,D,13,2017-08-16,0,0,0,0,0
3,3000891,2017-08-16,1,BEVERAGES,20,3,0,2017,8,16,...,Quito,Pichincha,D,13,2017-08-16,0,0,0,0,0
4,3000892,2017-08-16,1,BOOKS,0,3,0,2017,8,16,...,Quito,Pichincha,D,13,2017-08-16,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,4,0,2017,8,31,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
28508,3029396,2017-08-31,9,PREPARED FOODS,0,4,0,2017,8,31,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
28509,3029397,2017-08-31,9,PRODUCE,1,4,0,2017,8,31,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,4,0,2017,8,31,...,Quito,Pichincha,B,6,2017-08-31,0,0,0,0,0


# Exercise 2: Hybrid Models 

To create the sales by store number for all years, not just 2017. 

In [8]:
store_sales = pd.read_csv(
    'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()

In [9]:
store_sales

sales  onpromotion
store_nbr family     date                              
1         AUTOMOTIVE 2013-01-01   0.000000            0
                     2013-01-02   2.000000            0
                     2013-01-03   3.000000            0
                     2013-01-04   3.000000            0
                     2013-01-05   5.000000            0
...                                    ...          ...
9         SEAFOOD    2017-08-11  23.830999            0
                     2017-08-12  16.859001            4
                     2017-08-13  20.000000            0
                     2017-08-14  17.000000            0
                     2017-08-15  16.000000            0

[3000888 rows x 2 columns]

To create a dataframe for average sales by family and then promotions by family, although I'm not sure what the promotions by family signify. 

In [10]:
family_sales = (
    store_sales
    .groupby(['family', 'date'])
    .mean()
    .unstack('family')
    .loc['2017']
)

In [11]:
family_sales

sales                                              \
family     AUTOMOTIVE BABY CARE     BEAUTY    BEVERAGES     BOOKS   
date                                                                
2017-01-01   0.092593  0.037037   0.055556    74.222221  0.000000   
2017-01-02  11.481482  0.259259  11.648149  6208.055664  0.481481   
2017-01-03   8.296296  0.296296   7.185185  4507.814941  0.814815   
2017-01-04   6.833333  0.333333   6.888889  3911.833252  0.759259   
2017-01-05   6.333333  0.351852   5.925926  3258.796387  0.407407   
...               ...       ...        ...          ...       ...   
2017-08-11   8.166667  0.129630   6.500000  3502.055664  0.000000   
2017-08-12   7.462963  0.055556   6.833333  3376.259277  0.000000   
2017-08-13   8.907408  0.166667   8.018518  3747.296387  0.018519   
2017-08-14   5.407407  0.166667   6.240741  3237.629639  0.000000   
2017-08-15   6.240741  0.148148   6.277778  3162.462891  0.000000   

                                                                           \
family     BREAD/BAKERY CELEBRATION     CLEANING        DAIRY        DELI   
date                                                                        
2017-01-01     9.084685    0.129630     7.500000    11.518518    3.629167   
2017-01-02   844.836304   14.203704  2233.648193  1545.000000  539.114807   
2017-01-03   665.124084   10.629630  1711.907349  1204.203735  404.300079   
2017-01-04   594.160583   11.185185  1508.036987  1107.796265  309.397675   
2017-01-05   495.511597   12.444445  1241.833374   829.277771  260.776489   
...                 ...         ...          ...          ...         ...   
2017-08-11   522.305298   16.111111  1031.148193   831.648132  345.934967   
2017-08-12   507.473114   15.722222  1072.870361   793.592590  290.553589   
2017-08-13   615.976990   10.074074  1143.648193   928.000000  325.801361   
2017-08-14   518.564026   11.000000  1019.111084   797.222229  271.532227   
2017-08-15   539.966553   11.055555  1082.851807   753.833313  276.582458   

            ... onpromotion                                        \
family      ...   MAGAZINES      MEATS PERSONAL CARE PET SUPPLIES   
date        ...                                                     
2017-01-01  ...         0.0   0.018519      0.111111     0.018519   
2017-01-02  ...         0.0   0.462963     10.592593     0.537037   
2017-01-03  ...         0.0   0.481481      9.722222     0.444444   
2017-01-04  ...         0.0   0.370370     12.037037     0.444444   
2017-01-05  ...         0.0   8.981481      5.666667     0.000000   
...         ...         ...        ...           ...          ...   
2017-08-11  ...         0.0  22.925926      9.000000     0.000000   
2017-08-12  ...         0.0   0.000000      8.703704     0.000000   
2017-08-13  ...         0.0   0.000000      9.240741     0.000000   
2017-08-14  ...         0.0   0.000000      8.722222     0.000000   
2017-08-15  ...         0.0   0.000000      8.814815     0.000000   

                                                                          \
family     PLAYERS AND ELECTRONICS    POULTRY PREPARED FOODS     PRODUCE   
date                                                                       
2017-01-01                0.000000   0.000000       0.037037    0.129630   
2017-01-02                0.000000   0.259259       1.166667    5.629630   
2017-01-03                0.000000   0.388889       1.351852   56.296296   
2017-01-04                0.000000   0.296296       5.444444  101.277778   
2017-01-05                0.000000   0.296296       0.907407    5.018519   
...                            ...        ...            ...         ...   
2017-08-11                0.000000  27.407407       0.537037    6.000000   
2017-08-12                0.018519   0.111111       0.277778    6.000000   
2017-08-13                0.000000   0.185185       0.222222    5.907407   
2017-08-14                0.000000   0.129630       0.148148    6.018519   
2017-08-15     

To create a new class to use for the boosted hybrid model

In [12]:
class BoostedHybrid:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2
        self.y_columns = None  # store column names from fit method

To create a method to fit two X datasets against one y dataset. Each X is for one of the models. 

In [13]:
def fit(self, X_1, X_2, y):
    self.model_1.fit(X_1, y)

    y_fit = pd.DataFrame(
        self.model_1.predict(X_1),
        index=X_1.index, columns=y.columns,
    )

    # Compute residuals
    y_resid = y - y_fit
    y_resid = y_resid.stack().squeeze() # wide to long

    # Fit self.model_2 on residuals
    self.model_2.fit(X_2, y_resid)

    # Save column names for predict method
    self.y_columns = y.columns
    # Save data for question checking
    self.y_fit = y_fit
    self.y_resid = y_resid


# Add method to class
BoostedHybrid.fit = fit


Now define the predict for the 2 X values. I'm not sure how the stack and unstack lines work. 

In [14]:
def predict(self, X_1, X_2):
    y_pred = pd.DataFrame(
        self.model_1.predict(X_1),
        index=X_1.index, columns=self.y_columns,
    )
    y_pred = y_pred.stack().squeeze()  # wide to long

    y_pred += self.model_2.predict(X_2)
    
    return y_pred.unstack()  # long to wide


# Add method to class
BoostedHybrid.predict = predict

To create a y value for all the average sales for each product for each day of 2017. 

In [15]:
y = family_sales.loc[:, 'sales']

In [16]:
y

family,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,DELI,...,MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,0.092593,0.037037,0.055556,74.222221,0.000000,9.084685,0.129630,7.500000,11.518518,3.629167,...,0.074074,5.013166,2.629630,0.074074,0.222222,2.855537,1.040722,31.163778,0.000000,0.259259
2017-01-02,11.481482,0.259259,11.648149,6208.055664,0.481481,844.836304,14.203704,2233.648193,1545.000000,539.114807,...,7.777778,488.522339,714.370361,12.814815,22.777779,581.413147,153.555542,3409.500488,2.925926,33.581944
2017-01-03,8.296296,0.296296,7.185185,4507.814941,0.814815,665.124084,10.629630,1711.907349,1204.203735,404.300079,...,3.500000,427.520905,488.333344,8.870370,13.555555,462.013672,125.820534,3394.559814,2.018518,27.135260
2017-01-04,6.833333,0.333333,6.888889,3911.833252,0.759259,594.160583,11.185185,1508.036987,1107.796265,309.397675,...,3.500000,369.121490,426.574066,8.129630,12.092592,431.276398,116.740814,3467.447998,1.722222,22.034130
2017-01-05,6.333333,0.351852,5.925926,3258.796387,0.407407,495.511597,12.444445,1241.833374,829.277771,260.776489,...,3.203704,368.945679,360.925934,7.629630,10.333333,345.673492,101.434570,2069.316650,1.425926,20.615334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,8.166667,0.129630,6.500000,3502.055664,0.000000,522.305298,16.111111,1031.148193,831.648132,345.934967,...,9.259259,498.964661,295.888885,10.870370,12.111111,468.857361,96.286926,2198.854492,65.240738,23.566963
2017-08-12,7.462963,0.055556,6.833333,3376.259277,0.000000,507.473114,15.722222,1072.870361,793.592590,290.553589,...,8.944445,329.178711,294.500000,9.407408,13.185185,354.342773,84.693817,2070.154785,67.481483,19.037592
2017-08-13,8.907408,0.166667,8.018518,3747.296387,0.018519,615.976990,10.074074,1143.648193,928.000000,325.801361,...,8.685185,345.055817,336.814819,10.018518,13.722222,379.801208,91.509422,2331.922363,68.851852,20.704575


To create a trend X value for the linear programming model. 

In [17]:
dp = DeterministicProcess(index=y.index, order=1)
X_1 = dp.in_sample()

In [18]:
X_1

,trend
date,
2017-01-01,1.0
2017-01-02,2.0
2017-01-03,3.0
2017-01-04,4.0
2017-01-05,5.0
...,...
2017-08-11,223.0
2017-08-12,224.0
2017-08-13,225.0


To create the X factor for XGBoost, with just the onpromotion feature utilized. 

In [19]:
X_2 = family_sales.drop('sales', axis=1).stack()

In [20]:
X_2

onpromotion
date       family                                 
2017-01-01 AUTOMOTIVE                     0.000000
           BABY CARE                      0.000000
           BEAUTY                         0.000000
           BEVERAGES                      0.703704
           BOOKS                          0.000000
...                                            ...
2017-08-15 POULTRY                        0.111111
           PREPARED FOODS                 0.166667
           PRODUCE                       58.685185
           SCHOOL AND OFFICE SUPPLIES     2.740741
           SEAFOOD                        0.000000

[7491 rows x 1 columns]

The label encoding for 'family' to change it from a categorical value to a numerical value. 

In [21]:
le = LabelEncoder()
X_2 = X_2.reset_index('family')
X_2['family'] = le.fit_transform(X_2['family'])

In [22]:
X_2

,family,onpromotion
date,,
2017-01-01,0,0.000000
2017-01-01,1,0.000000
2017-01-01,2,0.000000
2017-01-01,3,0.703704
2017-01-01,4,0.000000
...,...,...
2017-08-15,28,0.111111
2017-08-15,29,0.166667
2017-08-15,30,58.685185


To add the day of the month to the XGBoost X dataframe. 

In [23]:
X_2["day"] = X_2.index.day  

In [24]:
X_2

,family,onpromotion,day
date,,,
2017-01-01,0,0.000000,1
2017-01-01,1,0.000000,1
2017-01-01,2,0.000000,1
2017-01-01,3,0.703704,1
2017-01-01,4,0.000000,1
...,...,...,...
2017-08-15,28,0.111111,15
2017-08-15,29,0.166667,15
2017-08-15,30,58.685185,15


To define the model combining linear regression and XGBRegressor. 

In [25]:
model = BoostedHybrid(LinearRegression(fit_intercept=False), XGBRegressor())

model.fit(X_1, X_2, y)
y_pred = model.predict(X_1, X_2)

y_pred = y_pred.clip(0.0)

In [26]:
y_pred

family,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,DELI,...,MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01,5.153979,5.106370,5.141543,30.272480,5.105710,8.760529,5.101250,4.430238,0.000000,17.100396,...,4.929466,66.147739,14.408252,15.246856,8.699985,105.378399,0.000000,133.196875,0.000000,1.855846
2017-01-02,0.000000,5.107683,9.363442,5658.788755,5.106363,195.628053,9.375859,1720.985178,1225.152294,280.311414,...,4.972707,107.096350,354.042766,48.926488,8.777052,232.866052,122.686107,2694.659803,0.000000,15.572623
2017-01-03,7.072391,6.929562,11.220496,4370.230740,6.927582,71.927950,11.285620,1103.294357,1008.938707,185.046314,...,6.836515,110.579889,209.169572,50.008247,9.877468,187.029790,83.266694,3287.621663,1.057266,16.738031
2017-01-04,8.208842,8.018404,8.811626,3799.133663,8.015764,291.246982,12.462344,1339.691426,596.853905,170.108790,...,7.967282,114.127606,252.736335,51.154189,11.042064,138.249010,152.736716,3324.209255,2.231471,17.967614
2017-01-05,8.498277,8.260230,9.088625,2703.309779,8.256929,292.653125,2.207640,1069.324579,713.676085,30.465612,...,8.251034,66.694611,286.194272,11.266374,11.359642,127.656393,85.434094,1806.616182,2.318149,18.109669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,12.478059,1.861143,12.113112,3614.011141,1.713950,783.500085,23.962945,1443.377091,898.529635,556.262983,...,10.992023,787.212925,522.500760,17.361869,21.521635,665.403891,155.069444,2335.226364,47.525629,54.204141
2017-08-12,12.526981,1.862456,8.939096,3637.608888,1.714603,787.155557,24.052140,1957.995107,1338.576683,548.756138,...,11.035263,603.918041,524.615481,17.420282,21.598702,620.798636,161.785535,2350.903556,47.612307,42.605354
2017-08-13,11.729587,1.017452,8.129264,3832.418549,0.868939,1144.616503,23.295017,1793.284991,1144.788557,572.950907,...,10.232186,605.531917,437.167790,16.632377,20.829450,640.339790,161.566131,3032.931028,46.852668,27.821864


In [27]:
mean_squared_log_error(y,y_pred)

0.47065321714293523

In [28]:
store_sales_prep = store_sales.drop('onpromotion', axis=1)

In [29]:
store_sales_prep

sales
store_nbr family     date                 
1         AUTOMOTIVE 2013-01-01   0.000000
                     2013-01-02   2.000000
                     2013-01-03   3.000000
                     2013-01-04   3.000000
                     2013-01-05   5.000000
...                                    ...
9         SEAFOOD    2017-08-11  23.830999
                     2017-08-12  16.859001
                     2017-08-13  20.000000
                     2017-08-14  17.000000
                     2017-08-15  16.000000

[3000888 rows x 1 columns]

In [30]:
y_prep = store_sales_prep.unstack(['store_nbr', 'family']).loc["2017"]

In [31]:
y_prep

sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-01-01        0.0       0.0    0.0       0.0   0.0     0.000000   
2017-01-02        5.0       0.0    0.0    1434.0   0.0   166.819000   
2017-01-03        4.0       0.0    4.0    3081.0   2.0   519.348022   
2017-01-04        1.0       0.0    4.0    3039.0   2.0   543.250977   
2017-01-05        2.0       0.0    3.0    2617.0   0.0   533.479980   
...               ...       ...    ...       ...   ...          ...   
2017-08-11        1.0       0.0    1.0    1006.0   0.0   145.606995   
2017-08-12        6.0       0.0    3.0    1659.0   0.0   243.220001   
2017-08-13        1.0       0.0    1.0     803.0   0.0   136.679001   
2017-08-14        1.0       0.0    6.0    2201.0   0.0   346.037994   
2017-08-15        4.0       0.0    4.0    1942.0   0.0   329.541016   

                                                     ...            \
store_nbr                                            ...         9   
family     CELEBRATION CLEANING   DAIRY        DELI  ... MAGAZINES   
date                                                 ...             
2017-01-01         0.0      0.0     0.0    0.000000  ...       0.0   
2017-01-02         0.0    332.0   376.0   44.980000  ...       5.0   
2017-01-03        15.0    952.0  1045.0  209.300003  ...       2.0   
2017-01-04        17.0   1055.0  1029.0  135.944000  ...       3.0   
2017-01-05        40.0    918.0   853.0  137.005997  ...       2.0   
...                ...      ...     ...         ...  ...       ...   
2017-08-11         4.0    341.0   343.0   64.302002  ...       5.0   
2017-08-12         3.0    351.0   526.0   99.487999  ...       2.0   
2017-08-13         1.0    169.0   266.0   47.770000  ...       3.0   
2017-08-14         4.0    571.0   699.0  154.578003  ...      12.0   
2017-08-15        21.0    703.0   602.0  116.402000  ...      11.0   

                                                                           \
store_nbr                                                                   
family           MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                        
2017-01-01    0.000000           0.0          0.0                     0.0   
2017-01-02  659.570007        1243.0         11.0                    41.0   
2017-01-03  547.364014         876.0          6.0                    15.0   
2017-01-04  395.287994         677.0          6.0                    13.0   
2017-01-05  470.768005         604.0          7.0                    10.0   
...                ...           ...          ...                     ...   
2017-08-11  309.244995         373.0         11.0                     2.0   
2017-08-12  260.298004         400.0          7.0                    10.0   
2017-08-13  327.205994         510.0          2.0                     9.0   
2017-08-14  330.975006         445.0          2.0                    14.0   
2017-08-15  449.227997         522.0          6.0                     6.0   

                                                                               \
store_nbr                                                                       
family         POULTRY PREPARED FOODS      PRODUCE SCHOOL AND OFFICE SUPPLIES   
date                                                                            
2017-01-01    0.000000       0.000000     0.000000                        0.0   
2017-01-02  843.596008     115.188995  3136.895996                        1.0   
2017-01-03  714.659973     133.039001  3229.558105                        1.0   
2017-01-04  536.830017      75.201004  1491.416992                        7.0   
2017-01-05  414.100006     113.698997  1566.821045                        1.0   
...           

In [32]:
model = BoostedHybrid(LinearRegression(fit_intercept=False), XGBRegressor())

model.fit(X_1, X_2, y_prep)
y_pred = model.predict(X_1, X_2)

y_pred = y_pred.clip(0.0)

In [33]:
y_pred

sales                                              \
store_nbr           1                                               
family     AUTOMOTIVE BABY CARE     BEAUTY    BEVERAGES     BOOKS   
date                                                                
2017-01-01   0.000000  0.000000   0.000000    10.548725  2.396941   
2017-01-02   0.000000  0.000000   3.057951  1346.656127  2.399198   
2017-01-03   5.196943  2.119076   1.324664  2786.010741  6.185059   
2017-01-04   4.486724  1.380449   7.886149  2849.096434  2.397282   
2017-01-05   6.099514  1.380449   9.492153  2079.391356  2.399538   
...               ...       ...        ...          ...       ...   
2017-08-11   5.505688  0.000000   6.970515  1597.027159  1.357816   
2017-08-12   5.534095  0.000000   6.448590  1967.142637  1.360073   
2017-08-13   5.562502  0.000000  10.551252  1473.433286  1.362330   
2017-08-14   5.397211  0.000000  10.922723  2166.274717  1.170887   
2017-08-15   5.425618  0.000000  10.400798  2202.398435  1.173144   

                                                                         ...  \
store_nbr                                                                ...   
family     BREAD/BAKERY CELEBRATION    CLEANING       DAIRY        DELI  ...   
date                                                                     ...   
2017-01-01     8.704517    2.492772    2.579921   22.496998    0.000000  ...   
2017-01-02    59.602875    8.461819  412.058904  312.199696   64.761569  ...   
2017-01-03    61.927794    1.690518  476.864662  487.063735   80.464786  ...   
2017-01-04   330.494804    5.694886  722.175975  820.614175   68.725684  ...   
2017-01-05   185.200399    3.749150  447.639356  690.547550   14.355948  ...   
...                 ...         ...         ...         ...         ...  ...   
2017-08-11   505.894966   22.728108  668.713737  265.127007  142.043332  ...   
2017-08-12   508.219885   22.826195  682.796076  653.806537  221.312686  ...   
2017-08-13   389.499941   22.924283  691.182431  525.477949  231.503224  ...   
2017-08-14   648.287819   22.828671  883.918014  702.569954  184.354669  ...   
2017-08-15   580.626852   22.926759  999.419153  881.689043  213.842255  ...   

                                                              \
store_nbr           9                                          
family      MAGAZINES       MEATS PERSONAL CARE PET SUPPLIES   
date                                                           
2017-01-01   4.525074   95.249794     29.468325    26.361579   
2017-01-02   4.548871  235.829011    750.673433   116.685350   
2017-01-03   6.690299  197.036462    397.648577    60.401992   
2017-01-04   9.883440  202.795549    303.449679    63.622963   
2017-01-05  11.995837  236.107904    335.566181    22.954905   
...               ...         ...           ...          ...   
2017-08-11   6.218524  603.305299    758.110293    15.206579   
2017-08-12   6.242320  619.487325    699.117626    15.258204   
2017-08-13   2.703172  618.514127    676.935566    11.746884   
2017-08-14   0.000000  617.524225    720.775544     8.218861   
2017-08-15   6.535352  627.478205    711.549158    15.634722   

                                                                            \
store_nbr                                                                    
family     PLAYERS AND ELECTRONICS     POULTRY PREPARED FOODS      PRODUCE   
date                                                                         
2017-01-01               26.382714  116.467177      24.947480   212.594920   
2017-01-02               26.455475  196.828534     136.965902  2357.113881   
2017-01-03               19.745717  192.966183      96.384087  3032.783737   
2017-01-04               22.987822  199.055705     130.038997  1227.754007   
2017-01-05               23.060582  149.229410     126.903318  1493.287755   
...                            ...         ...            ...          ...   
2017-08-11               19.919782  77

In [34]:
df_test = pd.read_csv(
    'test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

In [35]:
df_test

id  onpromotion
store_nbr family     date                            
1         AUTOMOTIVE 2017-08-16  3000888            0
                     2017-08-17  3002670            0
                     2017-08-18  3004452            0
                     2017-08-19  3006234            0
                     2017-08-20  3008016            0
...                                  ...          ...
9         SEAFOOD    2017-08-27  3022271            0
                     2017-08-28  3024053            0
                     2017-08-29  3025835            0
                     2017-08-30  3027617            0
                     2017-08-31  3029399            0

[28512 rows x 2 columns]

In [36]:
family_sales_test = (
    df_test
    .groupby(['family', 'date'])
    .mean()
    .unstack('family')
    .loc['2017']
)

In [37]:
family_sales_test

id                                              \
family     AUTOMOTIVE  BABY CARE     BEAUTY  BEVERAGES      BOOKS   
date                                                                
2017-08-16  3001762.5  3001763.5  3001764.5  3001765.5  3001766.5   
2017-08-17  3003544.5  3003545.5  3003546.5  3003547.5  3003548.5   
2017-08-18  3005326.5  3005327.5  3005328.5  3005329.5  3005330.5   
2017-08-19  3007108.5  3007109.5  3007110.5  3007111.5  3007112.5   
2017-08-20  3008890.5  3008891.5  3008892.5  3008893.5  3008894.5   
2017-08-21  3010672.5  3010673.5  3010674.5  3010675.5  3010676.5   
2017-08-22  3012454.5  3012455.5  3012456.5  3012457.5  3012458.5   
2017-08-23  3014236.5  3014237.5  3014238.5  3014239.5  3014240.5   
2017-08-24  3016018.5  3016019.5  3016020.5  3016021.5  3016022.5   
2017-08-25  3017800.5  3017801.5  3017802.5  3017803.5  3017804.5   
2017-08-26  3019582.5  3019583.5  3019584.5  3019585.5  3019586.5   
2017-08-27  3021364.5  3021365.5  3021366.5  3021367.5  3021368.5   
2017-08-28  3023146.5  3023147.5  3023148.5  3023149.5  3023150.5   
2017-08-29  3024928.5  3024929.5  3024930.5  3024931.5  3024932.5   
2017-08-30  3026710.5  3026711.5  3026712.5  3026713.5  3026714.5   
2017-08-31  3028492.5  3028493.5  3028494.5  3028495.5  3028496.5   

                                                                      ...  \
family     BREAD/BAKERY CELEBRATION   CLEANING      DAIRY       DELI  ...   
date                                                                  ...   
2017-08-16    3001767.5   3001768.5  3001769.5  3001770.5  3001771.5  ...   
2017-08-17    3003549.5   3003550.5  3003551.5  3003552.5  3003553.5  ...   
2017-08-18    3005331.5   3005332.5  3005333.5  3005334.5  3005335.5  ...   
2017-08-19    3007113.5   3007114.5  3007115.5  3007116.5  3007117.5  ...   
2017-08-20    3008895.5   3008896.5  3008897.5  3008898.5  3008899.5  ...   
2017-08-21    3010677.5   3010678.5  3010679.5  3010680.5  3010681.5  ...   
2017-08-22    3012459.5   3012460.5  3012461.5  3012462.5  3012463.5  ...   
2017-08-23    3014241.5   3014242.5  3014243.5  3014244.5  3014245.5  ...   
2017-08-24    3016023.5   3016024.5  3016025.5  3016026.5  3016027.5  ...   
2017-08-25    3017805.5   3017806.5  3017807.5  3017808.5  3017809.5  ...   
2017-08-26    3019587.5   3019588.5  3019589.5  3019590.5  3019591.5  ...   
2017-08-27    3021369.5   3021370.5  3021371.5  3021372.5  3021373.5  ...   
2017-08-28    3023151.5   3023152.5  3023153.5  3023154.5  3023155.5  ...   
2017-08-29    3024933.5   3024934.5  3024935.5  3024936.5  3024937.5  ...   
2017-08-30    3026715.5   3026716.5  3026717.5  3026718.5  3026719.5  ...   
2017-08-31    3028497.5   3028498.5  3028499.5  3028500.5  3028501.5  ...   

           onpromotion                                        \
family       MAGAZINES      MEATS PERSONAL CARE PET SUPPLIES   
date                                                           
2017-08-16    0.000000   0.018519     18.000000     0.000000   
2017-08-17    0.000000   8.981481      8.740741     0.018519   
2017-08-18    0.000000  22.796296      9.907407     0.000000   
2017-08-19    0.000000   0.037037     10.722222     0.000000   
2017-08-20    0.000000   0.000000     11.203704     0.000000   
2017-08-21    0.000000   0.055556     10.314815     0.000000   
2017-08-22    0.000000   0.000000     10.185185     0.000000   
2017-08-23    0.000000   0.000000     12.018519     0.018519   
2017-08-24    0.000000   9.203704      9.648148     0.000000   
2017-08-25    0.000000  23.092593      9.796296     0.000000   
2017-08-26    0.000000   0.018519     11.518519     0.000000   
2017-08-27    0.000000   0.037037     12.648148     0.000000   
2017-08-28    0.000000   0.000000     11.092593     0.000000   
2017-08-29    0.000000   0.018519     10.574074     0.000000   
2017-08-30    0.074074   0.981481     13.111111     0.055556   
2017-08-31    0.037037   9.388889     11.611111     0.037037   

                          

In [38]:
y_prep = store_sales.unstack(['store_nbr', 'family']).loc["2017"]

In [39]:
y_prep

sales                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-01-01        0.0       0.0    0.0       0.0   0.0     0.000000   
2017-01-02        5.0       0.0    0.0    1434.0   0.0   166.819000   
2017-01-03        4.0       0.0    4.0    3081.0   2.0   519.348022   
2017-01-04        1.0       0.0    4.0    3039.0   2.0   543.250977   
2017-01-05        2.0       0.0    3.0    2617.0   0.0   533.479980   
...               ...       ...    ...       ...   ...          ...   
2017-08-11        1.0       0.0    1.0    1006.0   0.0   145.606995   
2017-08-12        6.0       0.0    3.0    1659.0   0.0   243.220001   
2017-08-13        1.0       0.0    1.0     803.0   0.0   136.679001   
2017-08-14        1.0       0.0    6.0    2201.0   0.0   346.037994   
2017-08-15        4.0       0.0    4.0    1942.0   0.0   329.541016   

                                                     ... onpromotion        \
store_nbr                                            ...           9         
family     CELEBRATION CLEANING   DAIRY        DELI  ...   MAGAZINES MEATS   
date                                                 ...                     
2017-01-01         0.0      0.0     0.0    0.000000  ...           0     0   
2017-01-02         0.0    332.0   376.0   44.980000  ...           0     0   
2017-01-03        15.0    952.0  1045.0  209.300003  ...           0     0   
2017-01-04        17.0   1055.0  1029.0  135.944000  ...           0     0   
2017-01-05        40.0    918.0   853.0  137.005997  ...           0    21   
...                ...      ...     ...         ...  ...         ...   ...   
2017-08-11         4.0    341.0   343.0   64.302002  ...           0     0   
2017-08-12         3.0    351.0   526.0   99.487999  ...           0     0   
2017-08-13         1.0    169.0   266.0   47.770000  ...           0     0   
2017-08-14         4.0    571.0   699.0  154.578003  ...           0     0   
2017-08-15        21.0    703.0   602.0  116.402000  ...           0     0   

                                                                       \
store_nbr                                                               
family     PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS POULTRY   
date                                                                    
2017-01-01             0            0                       0       0   
2017-01-02            13            0                       0       2   
2017-01-03            11            0                       0       1   
2017-01-04            15            0                       0       1   
2017-01-05             8            0                       0       1   
...                  ...          ...                     ...     ...   
2017-08-11            11            0                       0      22   
2017-08-12             7            0                       0       0   
2017-08-13             9            0                       0       0   
2017-08-14            10            0                       0       0   
2017-08-15            11            0                       0       0   

                                                                      
store_nbr                                                             
family     PREPARED FOODS PRODUCE SCHOOL AND OFFICE SUPPLIES SEAFOOD  
date                                                                  
2017-01-01              0       0                          0       0  
2017-01-02              1       4                          0       0  
2017-01-03              2     150                          0       0  
2017-01-04              8       9                          0       0  
2017-01-05              1       5                          0       1  
...                   ...     ...           

In [40]:
dp = DeterministicProcess(index=family_sales_test.index, order=1)
X_1_submit = dp.in_sample()

In [41]:
X_1_submit

,trend
date,
2017-08-16,1.0
2017-08-17,2.0
2017-08-18,3.0
2017-08-19,4.0
2017-08-20,5.0
2017-08-21,6.0
2017-08-22,7.0
2017-08-23,8.0
2017-08-24,9.0


In [42]:
X_2_submit = family_sales_test.drop('id', axis=1).stack()

In [43]:
X_2_submit

onpromotion
date       family                                 
2017-08-16 AUTOMOTIVE                     0.000000
           BABY CARE                      0.000000
           BEAUTY                         2.000000
           BEVERAGES                     31.592593
           BOOKS                          0.000000
...                                            ...
2017-08-31 POULTRY                        0.388889
           PREPARED FOODS                 0.111111
           PRODUCE                        3.351852
           SCHOOL AND OFFICE SUPPLIES     2.870370
           SEAFOOD                        0.351852

[528 rows x 1 columns]

In [44]:
le = LabelEncoder()
X_2_submit = X_2_submit.reset_index('family')
X_2_submit['family'] = le.fit_transform(X_2_submit['family'])

In [45]:
X_2_submit

,family,onpromotion
date,,
2017-08-16,0,0.000000
2017-08-16,1,0.000000
2017-08-16,2,2.000000
2017-08-16,3,31.592593
2017-08-16,4,0.000000
...,...,...
2017-08-31,28,0.388889
2017-08-31,29,0.111111
2017-08-31,30,3.351852


In [46]:
X_2_submit["day"] = X_2_submit.index.day  

In [47]:
X_2_submit

,family,onpromotion,day
date,,,
2017-08-16,0,0.000000,16
2017-08-16,1,0.000000,16
2017-08-16,2,2.000000,16
2017-08-16,3,31.592593,16
2017-08-16,4,0.000000,16
...,...,...,...
2017-08-31,28,0.388889,31
2017-08-31,29,0.111111,31
2017-08-31,30,3.351852,31


In [96]:
#y_submit = pd.DataFrame(model.predict(X_1_submit, X_2_submit), index=X_1_submit.index, columns=y.columns)

In [48]:
y_submit = pd.DataFrame(model.predict(X_1_submit, X_2_submit), index=X_1_submit.index, columns=y_prep.columns)

In [50]:
y_submit

sales                                              \
store_nbr           1                                               
family     AUTOMOTIVE BABY CARE     BEAUTY    BEVERAGES     BOOKS   
date                                                                
2017-08-16  -0.994442 -1.022849  16.293124  1968.021240  0.663139   
2017-08-17   6.113851  6.057036  12.043701   638.317565  8.553826   
2017-08-18   1.352686  1.267464  10.193163   657.230224  3.766509   
2017-08-19  -1.212298 -1.325927   7.077847  1014.834777  1.175377   
2017-08-20   0.891885 -0.119952   8.380675  1314.292235  2.383608   
2017-08-21   0.050492 -0.119952   8.402297  1438.566405  2.385864   
2017-08-22   0.078899 -0.119952  10.093356   550.644316  2.388121   
2017-08-23  -5.243748 -5.471007   2.518259   390.310484 -5.157404   
2017-08-24   7.197053  4.633850   8.726172  -505.495364  5.848060   
2017-08-25  -0.074120 -0.358193   2.166239   -55.003589 -0.903684   
2017-08-26  -1.586617 -1.899098  -3.362544   519.097897 -2.442332   
2017-08-27  -0.472711 -0.813599  -1.480165   597.399960 -1.354576   
2017-08-28   2.275795 -0.491471  -7.316654   556.741848 -1.030191   
2017-08-29  -0.093768 -0.491471  -7.295031   513.849972 -1.027935   
2017-08-30  -8.453674 -6.368693 -13.855162   679.744411 -6.902899   
2017-08-31   6.040505  3.944833   0.632227   937.700862  3.412882   

                                                                         ...  \
store_nbr                                                                ...   
family     BREAD/BAKERY CELEBRATION    CLEANING       DAIRY        DELI  ...   
date                                                                     ...   
2017-08-16   413.246733    0.758970   23.959231  739.935145 -128.775601  ...   
2017-08-17   199.354473    8.745488  548.215215  560.710133   45.549194  ...   
2017-08-18   198.341349    4.054002  454.703713  500.280166  327.297908  ...   
2017-08-19   194.159126    2.571336  413.914256   80.192976   41.480338  ...   
2017-08-20  -139.372034    2.862762   71.653261  379.004764   43.632594  ...   
2017-08-21   154.743672    2.960850  408.114517  378.537108   44.503645  ...   
2017-08-22   190.796801    3.928738  470.612872  955.478081   45.374697  ...   
2017-08-23   296.800400   -2.083219  128.084997 -705.668927   52.130033  ...   
2017-08-24   238.254515    6.710538  453.035912  182.651685   49.347543  ...   
2017-08-25   292.098278    0.537851  392.601985  135.823806 -422.738704  ...   
2017-08-26   291.529429   -0.904966  -49.807406   83.658304    9.898034  ...   
2017-08-27    15.508295    0.278621   64.345557  105.981728    2.238211  ...   
2017-08-28   194.318182    0.698836  369.150210  107.347380   24.230296  ...   
2017-08-29   267.893009    0.478029  997.127660  113.250146   25.101348  ...   
2017-08-30   169.192965  -10.281642  -39.297302 -417.896229   44.093630  ...   
2017-08-31   194.001740    3.870467 -128.800298  637.885046   59.469268  ...   

           onpromotion                                   \
store_nbr            9                                    
family       MAGAZINES MEATS PERSONAL CARE PET SUPPLIES   
date                                                      
2017-08-16         NaN   NaN           NaN          NaN   
2017-08-17         NaN   NaN           NaN          NaN   
2017-08-18         NaN   NaN           NaN          NaN   
2017-08-19         NaN   NaN           NaN          NaN   
2017-08-20         NaN   NaN           NaN          NaN   
2017-08-21         NaN   NaN           NaN          NaN   
2017-08-22         NaN   NaN           NaN          NaN   
2017-08-23         NaN   NaN           NaN          NaN   
2017-08-24         NaN   NaN           NaN          NaN   
2017-08-25         NaN   NaN           NaN          NaN   
2017-08-26         NaN   NaN           NaN          NaN   
2017-08-27         NaN   NaN           NaN          NaN   
2017-08-28         NaN   NaN           NaN          NaN   
2017-08-29         NaN   NaN   

In [51]:
y_submit = y_submit.stack(['store_nbr', 'family'])

In [52]:
y_submit['sales'] = [a if a>0 else 0 for a in y_submit['sales']]

In [53]:
y_submit

onpromotion        sales
date       store_nbr family                                              
2017-08-16 1         AUTOMOTIVE                          NaN     0.000000
                     BABY CARE                           NaN     0.000000
                     BEAUTY                              NaN    16.293124
                     BEVERAGES                           NaN  1968.021240
                     BOOKS                               NaN     0.663139
...                                                      ...          ...
2017-08-31 9         POULTRY                             NaN   132.482012
                     PREPARED FOODS                      NaN     1.832343
                     PRODUCE                             NaN   716.125099
                     SCHOOL AND OFFICE SUPPLIES          NaN   206.065567
                     SEAFOOD                             NaN     7.881995

[28512 rows x 2 columns]

In [54]:
y_submit = y_submit.join(df_test.id).reindex(columns=['id', 'sales'])

In [55]:
y_submit

id        sales
date       store_nbr family                                          
2017-08-16 1         AUTOMOTIVE                  3000888     0.000000
                     BABY CARE                   3000889     0.000000
                     BEAUTY                      3000890    16.293124
                     BEVERAGES                   3000891  1968.021240
                     BOOKS                       3000892     0.663139
...                                                  ...          ...
2017-08-31 9         POULTRY                     3029395   132.482012
                     PREPARED FOODS              3029396     1.832343
                     PRODUCE                     3029397   716.125099
                     SCHOOL AND OFFICE SUPPLIES  3029398   206.065567
                     SEAFOOD                     3029399     7.881995

[28512 rows x 2 columns]

In [56]:
y_submit.to_csv('./submissions/submit-exercise2.csv', index=False)

# End of Core Exercise - Looking at Using other Models

Depending on your problem, you might want to use other hybrid combinations than the linear regression + XGBoost hybrid you've created in the previous questions. Run the next cell to try other algorithms from scikit-learn.

In [ ]:
# Model 1 (trend)
from sklearn.linear_model import ElasticNet, Lasso, Ridge

# Model 2
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# Boosted Hybrid

# YOUR CODE HERE: Try different combinations of the algorithms above
model = BoostedHybrid(
    model_1=Ridge(),
    model_2=KNeighborsRegressor(),
)

These are just some suggestions. You might discover other algorithms you like in the scikit-learn User Guide.

Use the code in this cell to see the predictions your hybrid makes.

In [ ]:
y_train, y_valid = y[:"2017-07-01"], y["2017-07-02":]
X1_train, X1_valid = X_1[: "2017-07-01"], X_1["2017-07-02" :]
X2_train, X2_valid = X_2.loc[:"2017-07-01"], X_2.loc["2017-07-02":]

# Some of the algorithms above do best with certain kinds of
# preprocessing on the features (like standardization), but this is
# just a demo.
model.fit(X1_train, X2_train, y_train)
y_fit = model.predict(X1_train, X2_train).clip(0.0)
y_pred = model.predict(X1_valid, X2_valid).clip(0.0)

families = y.columns[0:6]
axs = y.loc(axis=1)[families].plot(
    subplots=True, sharex=True, figsize=(11, 9), **plot_params, alpha=0.5,
)
_ = y_fit.loc(axis=1)[families].plot(subplots=True, sharex=True, color='C0', ax=axs)
_ = y_pred.loc(axis=1)[families].plot(subplots=True, sharex=True, color='C3', ax=axs)
for ax, family in zip(axs, families):
    ax.legend([])
    ax.set_ylabel(family)